### Planning

Goal: Why are our customers churning?

*Specific questions to address*
- Does the month they signed up influence churn? 
    - plot the rate of churn on a line chart, where x is tenure and y is the customers churned/total customers
- Which, if any, features indicate a higher liklihood of churn?
- Is there a price threshold for likelihood of churn, specific to services? What is that point?
- What is the churn rate in the 13th month for one-year customers compared to month-to-month customers?

data_dictionary - variables included and why or not why? (doesn't have to be an actual python dictionary)

## Acquire

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import acquire
import split_scale

import warnings 
warnings.filterwarnings("ignore")

In [56]:
query = """
    SELECT * 
    FROM customers
    JOIN internet_service_types USING (internet_service_type_id)
    JOIN contract_types USING (contract_type_id)
    JOIN payment_types USING (payment_type_id)
    """
db = "telco_churn"

In [57]:
df = acquire.get_data(query, db)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
payment_type_id             7043 non-null int64
contract_type_id            7043 non-null int64
internet_service_type_id    7043 non-null int64
customer_id                 7043 non-null object
gender                      7043 non-null object
senior_citizen              7043 non-null int64
partner                     7043 non-null object
dependents                  7043 non-null object
tenure                      7043 non-null int64
phone_service               7043 non-null object
multiple_lines              7043 non-null object
online_security             7043 non-null object
online_backup               7043 non-null object
device_protection           7043 non-null object
tech_support                7043 non-null object
streaming_tv                7043 non-null object
streaming_movies            7043 non-null object
paperless_billing           7043 non-null object
monthly_charges 

In [59]:
df.describe()

,payment_type_id,contract_type_id,internet_service_type_id,senior_citizen,tenure,monthly_charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,2.315633,1.690473,1.872923,0.162147,32.371149,64.761692
std,1.148907,0.833755,0.737796,0.368612,24.559481,30.090047
min,1.000000,1.000000,1.000000,0.000000,0.000000,18.250000
25%,1.000000,1.000000,1.000000,0.000000,9.000000,35.500000
50%,2.000000,1.000000,2.000000,0.000000,29.000000,70.350000
75%,3.000000,2.000000,2.000000,0.000000,55.000000,89.850000
max,4.000000,3.000000,3.000000,1.000000,72.000000,118.750000


In [60]:
df.isnull().sum()

payment_type_id             0
contract_type_id            0
internet_service_type_id    0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
internet_service_type       0
contract_type               0
payment_type                0
dtype: int64

In [61]:
df.drop(columns = ["payment_type_id","contract_type_id","internet_service_type_id"], inplace=True)

In [62]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,No,No,Yes,No,59.9,542.4,No,DSL,Month-to-month,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,...,Yes,Yes,Yes,Yes,69.4,571.45,No,DSL,Month-to-month,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,...,No,No,No,Yes,48.2,340.35,No,DSL,Month-to-month,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,...,No,No,No,Yes,25.1,25.1,Yes,DSL,Month-to-month,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,...,No,No,No,No,30.5,30.5,Yes,DSL,Month-to-month,Bank transfer (automatic)


### Prepare

In [63]:
# split data using a train of 80% and a default random state of 123
train, test = split_scale.split_my_data(df, .80)

In [87]:
# split features and target
X_train = train.drop(columns="churn")
X_test = train.drop(columns="churn")
y_train = train[["churn"]]
y_test = test[["churn"]]

In [94]:
# change churn value from no/yes to 0/1
encoder = LabelEncoder()
encoder.fit(y_train.churn)
y_train.churn = encoder.transform(y_train.churn)
y_test.churn = encoder.transform(y_test.churn)

In [95]:
y_train.info()
y_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5634 entries, 6924 to 3582
Data columns (total 1 columns):
churn    5634 non-null int64
dtypes: int64(1)
memory usage: 88.0 KB


,churn
6924,0
5249,0
5852,0
3597,1
4147,0


In [96]:
X_train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,internet_service_type,contract_type,payment_type
6924,8148-NLEGT,Female,0,Yes,Yes,42,Yes,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,22.95,1014.25,None,Two year,Electronic check
5249,7244-KXYZN,Female,0,No,No,24,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.45,527.35,None,One year,Credit card (automatic)
5852,8106-GWQOK,Male,0,Yes,No,38,No,No phone service,Yes,Yes,No,Yes,Yes,Yes,Yes,60.00,2193.2,DSL,Two year,Bank transfer (automatic)
3597,4488-PSYCG,Male,0,No,No,16,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,No,20.25,327.45,None,Month-to-month,Bank transfer (automatic)
4147,4808-YNLEU,Female,0,Yes,No,35,Yes,No,Yes,No,No,No,Yes,No,Yes,62.15,2215.45,DSL,One year,Bank transfer (automatic)
